In [ ]:
pip install discord

In [ ]:
pip install scikit-fuzzy

In [ ]:
import tensorflow as tf
import discord
import os
import nest_asyncio
import requests
import json
import nltk
import random
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from nltk.tokenize import word_tokenize
from skfuzzy import control as ctrl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
nltk.download('punkt')
nest_asyncio.apply()

In [ ]:
#Load data CSV
data = pd.read_csv('HatchData.csv')

#Separate the input and output data
input_data = data[['Bulb_Data', 'Fan_Data', 'Humidity_Data', 'Temperature_Data']]
output_data = data['Valve_Data']

#Preprocess the input data using standardization
scaler = StandardScaler()
input_data = scaler.fit_transform(input_data)

#Create Keras model
model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train model
model.fit(input_data, output_data, epochs=50, batch_size=32)


In [ ]:
#Discord Bot Token
TOKEN = '#Enter Bot Token Here'
client = discord.Client(intents=discord.Intents.all())

In [ ]:
#TensorFlow Prediction
  #Creating a function to call it
def predictTensorflow(bulb,fan,humidity,temperature):
  inputData = [[bulb,fan,humidity,temperature]]
  inputData = scaler.transform(inputData)
  prediction = model.predict(inputData)
  return prediction[0][0]

In [ ]:
 #Fuzzy Logic
    #Creating a function to call it later
def generatePrediction(inputHumidity,inputTemperature):

      #Create antecedents and consequents
    temp = ctrl.Antecedent(np.arange(0, 101, 1), 'temperature')
    humid = ctrl.Antecedent(np.arange(0, 101, 1), 'humidity')
    incub = ctrl.Consequent(np.arange(0, 101, 1), 'incubation')

      #Define membership functions for temperature
    temp['cold'] = fuzz.trapmf(temp.universe, [0, 0, 20, 30])
    temp['cool'] = fuzz.trimf(temp.universe, [20, 30, 40])
    temp['moderate'] = fuzz.trimf(temp.universe, [30, 40, 50])
    temp['warm'] = fuzz.trimf(temp.universe, [40, 50, 60])
    temp['hot'] = fuzz.trapmf(temp.universe, [50, 60, 100, 100])

      #Define membership functions for humidity
    humid['dry'] = fuzz.trapmf(humid.universe, [0, 0, 20, 40])
    humid['low'] = fuzz.trimf(humid.universe, [20, 40, 60])
    humid['moderate'] = fuzz.trimf(humid.universe, [40, 60, 80])
    humid['high'] = fuzz.trapmf(humid.universe, [60, 80, 100, 100])

      #Define membership functions for success
    incub['fail'] = fuzz.trapmf(incub.universe, [0, 0, 40, 60])
    incub['success'] = fuzz.trapmf(incub.universe, [40, 60, 100, 100])

      #Define the rules
    rule1 = ctrl.Rule(temp['cold'] & humid['dry'], incub['fail'])
    rule2 = ctrl.Rule(temp['cool'] & humid['dry'], incub['fail'])
    rule3 = ctrl.Rule(temp['moderate'] & humid['dry'], incub['fail'])
    rule4 = ctrl.Rule(temp['warm'] & humid['dry'], incub['fail'])
    rule5 = ctrl.Rule(temp['hot'] & humid['dry'], incub['fail'])

    rule6 = ctrl.Rule(temp['cold'] & humid['low'], incub['fail'])
    rule7 = ctrl.Rule(temp['cool'] & humid['low'], incub['fail'])
    rule8 = ctrl.Rule(temp['moderate'] & humid['low'], incub['fail'])
    rule9 = ctrl.Rule(temp['warm'] & humid['low'], incub['success'])
    rule10 = ctrl.Rule(temp['hot'] & humid['low'], incub['fail'])

    rule11 = ctrl.Rule(temp['cold'] & humid['moderate'], incub['fail'])
    rule12 = ctrl.Rule(temp['cool'] & humid['moderate'], incub['fail'])
    rule13 = ctrl.Rule(temp['moderate'] & humid['moderate'], incub['success'])
    rule14 = ctrl.Rule(temp['warm'] & humid['moderate'], incub['success'])
    rule15 = ctrl.Rule(temp['hot'] & humid['moderate'], incub['fail'])

    rule16 = ctrl.Rule(temp['cold'] & humid['high'], incub['fail'])
    rule17 = ctrl.Rule(temp['cool'] & humid['high'], incub['success'])
    rule18 = ctrl.Rule(temp['moderate'] & humid['high'], incub['success'])
    rule19 = ctrl.Rule(temp['warm'] & humid['high'], incub['success'])
    rule20 = ctrl.Rule(temp['hot'] & humid['high'],incub['fail'])

      #Define the Control System with the rules
    egg_ctrl = ctrl.ControlSystem([rule1,rule2,rule3,rule4,rule5,rule6,rule7,rule8,rule9,rule10,rule11,rule12,rule13,rule14,rule15,rule16,rule17,rule18,rule19,rule20])

    eggHatch = ctrl.ControlSystemSimulation(egg_ctrl)
    eggHatch.input['temperature'] = inputTemperature
    eggHatch.input['humidity'] = inputHumidity

      #Compute the predictive measurement with the temperaturea and humidity provided
    eggHatch.compute()
    x = eggHatch.output['incubation']
    return x

    temperature.view()
    humidity.view()
    incubation.view()


In [ ]:
#Scrape json file
url = "https://eggincubator-825e1-default-rtdb.firebaseio.com/.json"
respond = requests.get(url)
data = json.loads(respond.text)

#Scrape specific data
bulbTime = data['Bulb on times:']
bulbStat = data['BulbStatus']
incubatorDay = data['Day']
incubatorFan = data['FanStatus']
incubatorHumidity = data['Humidity']
incubatorTemperature = data['Temperature']
valveStat = data['ValveStatus']

#Function to preprocess user input for Bag of Words NLP
def preprocess_input(input_text):
    tokens = nltk.word_tokenize(input_text.lower())
    return tokens

def generateAnswer(input_incubator):
    tokens = preprocess_input(input_incubator)
    for token in tokens:
      if token in incubator_responses:
        return random.choice(incubator_responses[token])
    return "Sorry, I don't understand."

#Bags of Words for different responses
incubator_responses = {
  'day': ['The Day is', 'Day is','Let it be known that the day is '],
  'temperature': ['The Temperatre is', 'Temperature is'],
  'humidity': ['The Humidity is', 'Humidity in the incubator is','Current Humidity is'],
  'fan': ['The incubator fan is ', 'Fan is', 'Currently the fan is '],
  'valve': ['The valve is now ', 'Valve is ','Currently the valve is ']
}

#Identify different types of egg
egg_types = {  #Dictionary of egg types and their characteristics
  'chicken': {
    'incubation_time': 21,
    'ideal_temperature': 37.5,
    'ideal_humidity': (50, 55),
    'turning_of_eggs': 3,
  },
  'duck': {
    'incubation_time': 28,
    'ideal_temperature': 37.5,
    'ideal_humidity': (60, 65),
    'turning_of_eggs': 4,
  },
  'goose': {
    'incubation_time': 28,
    'ideal_temperature': 37.5,
    'ideal_humidity': (70, 75),
    'turning_of_eggs': 5,
  }
}

class JacksonBot(discord.Client):

    async def on_ready(self):
        print(f'Logged in as {self.user}')
        print('Jackson Bot is Ready!')

      #Chatbot will greet once ready
        channel = self.get_channel()  #Channel ID HERE
        await channel.send("Hi, my name is Jackson.")
        await channel.send("How can I help you?")
        await channel.send("Tip 1: Ask me the real-time egg incubating conditions like temperature, humidity, fan, bulb status, bulb on time, day and valve.")
        await channel.send("Tip 2: Ask the condition for hatching a specific egg.")
        await channel.send("Tip 3: Ask the prediction of incubating the egg by providing the humidity and temperature. (Use 'predict' as keyword)")

    async def on_message(self,message):
        try:
               #Avoid the bot replying to ownself
            if message.author.id == self.user.id:
                return

              #Temperature Status Here
            if "temperature" in message.content.lower():
                incubatorInput = message.content.lower();
                incubatorRespond = generateAnswer(incubatorInput)
                await message.channel.send(f'{incubatorRespond} {incubatorTemperature} degrees Celsius.')

              #Humidity Status Here
            elif "humidity" in message.content.lower():
                incubatorInput = message.content.lower();
                incubatorRespond = generateAnswer(incubatorInput)
                await message.channel.send(f'{incubatorRespond} {incubatorHumidity}.')

              #Fan Status Here
            elif "fan" in message.content.lower():
                if incubatorFan == 0:
                  fanStat = "off"
                else:
                  fanStat = "on"
                incubatorInput = message.content.lower();
                incubatorRespond = generateAnswer(incubatorInput)
                await message.channel.send(f'{incubatorRespond} {fanStat}.')

              #Bulb Status Here
            elif "bulb status" in message.content.lower():
                if bulbStat == 0:
                  bulb_stat = "off"
                else:
                  bulb_stat = "on"
                messageBulbStat = ['The current bulb status is','The bulb status is','The bulb is']
                res = messageBulbStat[random.randint(0, 2)]
                await message.channel.send(f'{res.format(message)} {bulb_stat}.')

              #Bulb on Time Here
            elif "bulb on time" in message.content.lower():
                messageBulbTime = ['The current bulb time is','The bulb time is','The bulb has been on for']
                res = messageBulbTime[random.randint(0, 2)]
                await message.channel.send(f'{res.format(message)} {bulbTime} minutes.')

              #Incubator Day Here
            elif "day" in message.content.lower():
                incubatorInput = message.content.lower();
                incubatorRespond = generateAnswer(incubatorInput)
                await message.channel.send(f'{incubatorRespond} {incubatorDay}.')

              #Valve Status Here
            elif "valve" in message.content.lower():
                if valveStat == 0:
                  valve_stat = "off"
                else:
                  valve_stat = "on"
                incubatorInput = message.content.lower();
                incubatorRespond = generateAnswer(incubatorInput)
                await message.channel.send(f'{incubatorRespond}{valve_stat}.')

              #If user ask about egg
            elif "egg" in message.content.lower() and 'chicken' not in message.content.lower() and 'duck' not in message.content.lower() and 'goose' not in message.content.lower() and 'predict' not in message.content.lower():
                await message.channel.send('Currently I can provide information about three types of eggs: chicken egg, duck egg and goose egg. Which one you would like to know?')

              #Greeting message
            elif message.content.lower().startswith('hello') or message.content.lower().startswith('hi') or message.content.lower().startswith('halo') or message.content.lower().startswith('jackson'):
                greeting = ['Harlouu.', 'Hello Human!', 'Yo!', 'Yahallo! haha', 'Wasssupppp!']
                res = greeting[random.randint(0, 4)]
                await message.channel.send(res.format(message))

              #Who are you message
            elif message.content.lower().startswith('who are you') or message.content.lower().startswith('who you'):
                messageWho = ['I am Jackson!', 'Jackson is my name, an Egg Incubator Bot!', ' I am an Egg Incubator Bot!','Hello! I am Jackson!']
                res = messageWho[random.randint(0,3)]
                await message.channel.send(res.format(message))

              #How are you message
            elif message.content.lower().startswith('how are you') or  message.content.lower().startswith('are you okay'):
                messageHow = ['I am doing fine.', 'I am hanging in.', 'I am good.', 'I am A-okay! haha', 'Things are a bit tough here, but I will get through.']
                res = messageHow[random.randint(0, 4)]
                await message.channel.send(res.format(message))

              #Goodbye message
            elif message.content.lower().startswith('bye') or message.content.lower().startswith('goodbye'):
                messageBye = ['Byee, {0.author.mention}!', 'See you next time, {0.author.mention}!', 'It was a great pleasure talking to you, {0.author.mention}!', 'Adios, {0.author.mention}!']
                res = messageBye[random.randint(0, 3)]
                await message.channel.send(res.format(message))

              #thank you message
            elif (message.content.lower().startswith('thanks') or message.content.lower().startswith('thank')
                or message.content.lower().startswith('tq') or message.content.lower().startswith('3q') or message.content.lower().startswith('10q') or message.content.lower().startswith('thx')):
                messageBye = ['You are welcome, {0.author.mention}!', 'My pleasure, {0.author.mention}!']
                res = messageBye[random.randint(0, 1)]
                await message.channel.send(res.format(message))

              #Response to egg types
              #Chicken Egg
            elif "chicken" in message.content.lower() or "chicken egg" in message.content.lower() and 'predict' not in message.content.lower() and 'calculate' not in message.content.lower():
                egg_info = egg_types['chicken']
                await message.channel.send('To hatch chicken eggs:')
                await message.channel.send(f'- Incubate the eggs for {egg_info["incubation_time"]} days.')
                await message.channel.send(f'- Maintain a temperature of {egg_info["ideal_temperature"]} degrees Celsius.')
                await message.channel.send(f'- Maintain a humidity level between {egg_info["ideal_humidity"][0]}% and {egg_info["ideal_humidity"][1]}%.')
                await message.channel.send(f'- Turn the eggs {egg_info["turning_of_eggs"]} times per day.')

              #Duck Egg
            elif "duck" in message.content.lower() or "duck egg" in message.content.lower() and 'predict' not in message.content.lower() and 'calculate' not in message.content.lower():
                egg_info = egg_types['duck']
                await message.channel.send('To hatch duck eggs:')
                await message.channel.send(f'- Incubate the eggs for {egg_info["incubation_time"]} days.')
                await message.channel.send(f'- Maintain a temperature of {egg_info["ideal_temperature"]} degrees Celsius.')
                await message.channel.send(f'- Maintain a humidity level between {egg_info["ideal_humidity"][0]}% and {egg_info["ideal_humidity"][1]}%.')
                await message.channel.send(f'- Turn the eggs {egg_info["turning_of_eggs"]} times per day.')

              #Goose Egg
            elif "goose" in message.content.lower() or "goose egg" in message.content.lower() and 'predict' not in message.content.lower() and 'calculate' not in message.content.lower():
                egg_info = egg_types['goose']
                await message.channel.send('To hatch goose eggs:')
                await message.channel.send(f'- Incubate the eggs for {egg_info["incubation_time"]} days.')
                await message.channel.send(f'- Maintain a temperature of {egg_info["ideal_temperature"]} degrees Celsius.')
                await message.channel.send(f'- Maintain a humidity level between {egg_info["ideal_humidity"][0]}% and {egg_info["ideal_humidity"][1]}%.')
                await message.channel.send(f'- Turn the eggs {egg_info["turning_of_eggs"]} times per day.')

              #Prediction here
            elif "predict" in message.content.lower() or "success rate" in message.content.lower():
                await message.channel.send('Yes I can!')
                await message.channel.send('I have 2 types of prediction model: Fuzzy Logic & Tensorflow ')
                await message.channel.send('Please specify')

            elif "fuzzy logic" in message.content.lower():
                await message.channel.send('Please provide me the humidity: ')
                #Make sure the imput is from the one who wants to predict
                def check(m):
                    return m.author.id == message.author.id and m.channel == message.channel
                humi = await self.wait_for('message', check=check)
                inputHumidity = float(humi.content)

                await message.channel.send('Please provide me the temperature: ')
                #Make sure the imput is from the one who wants to predict
                def check(m):
                    return m.author.id == message.author.id and m.channel == message.channel
                tem = await self.wait_for('message', check=check)
                inputTemperature = float(tem.content)

                #Function call to prediction and round off to two-decimal points
                #To call : generatePredicition(variable/value,variable/value)
                prediction = round(generatePrediction(inputHumidity,inputTemperature),2)
                if prediction < 50:
                  await message.channel.send("Incubation FAILED with the rate of {}% ".format(prediction))
                else:
                  await message.channel.send("Incubation SUCCESSFUL with the rate of {}%".format(prediction))

            elif "tensorflow" in message.content.lower():
              await message.channel.send("Please specify if the bulb is ON or OFF")
              def check(m):
                return m.author.id == message.author.id and m.channel == message.channel
              bulb = await self.wait_for('message',check=check)
              inputBulb = bulb.content.lower()
              if "on" in inputBulb:
                  bulbData = 1
              else:
                  bulbData = 0
              await message.channel.send("Please specify if the fan is ON or OFF")
              def check(m):
                return m.author.id == message.author.id and m.channel == message.channel
              fan = await self.wait_for('message', check=check)
              inputFan = fan.content.lower()
              if "on" in inputFan:
                  fanData = 1
              else:
                  fanData = 0
              await message.channel.send("Please specify the humidity")
              def check(m):
                  return m.author.id == message.author.id and m.channel == message.channel
              humi = await self.wait_for('message', check=check)
              inputHumidity = float(humi.content)

              await message.channel.send("Please specify the temperature")
              def check(m):
                  return m.author.id == message.author.id and m.channel == message.channel
              tem = await self.wait_for('message', check=check)
              inputTemperature = float(tem.content)

              tensorflow = predictTensorflow(bulbData,fanData,inputHumidity,inputTemperature)*100
              await message.channel.send(f'Prediction: {tensorflow:.2f}% success rate for incubation')

            elif message.content.isdigit() or "on" in message.content.lower() or "off" in message.content.lower():
              return

            else:
              await message.channel.send("Sorry I do not understand")

              #Reaction with emoji
            if message.content == 'cool' or message.content == 'great':
                await message.add_reaction('\U0001F60E')
            if message.content == 'thank you':
                await message.add_reaction('\U0001F497')

              #Test for input from user in console
            print("Received message: " + message.content)

        except Exception as e:
            print(f"An Error occurred: {e}")
            await message.channel.send("Oops! An error occurred. Please try again later.")


    async def on_message_edit(self, before, after):
        await before.channel.send(
          f'{before.author} edit a message.\n'
          f'Before: {before.content}\n'
          f'After: {after.content}\n'
          )



intents = discord.Intents.default()
intents.members = True
intents.message_content = True
def main():
    client = JacksonBot(intents=intents)
    client.run(TOKEN)

if __name__ == '__main__':
    main()